In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import h5py
import config as cfg

seed = cfg.config_set['seed']

torch.manual_seed(seed)
np.random.seed(seed)

In [2]:
class LTPDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def count_parameters(model):
    return sum(p.numel() for p in model.parameters())



###* Prepare data

with open(cfg.config_set["data_path"]) as f:
    data = f.readlines()


for i in range(len(data)):
    data[i] = data[i].split()
    data[i] = [float(x) for x in data[i]]
    
data = np.array(data)

ratio_test_val_train = cfg.config_set["ratio_test_val_train"]

X_train, X_temp, y_train, y_temp = train_test_split(data[:, 0:3], data[:, 3:], test_size=ratio_test_val_train, random_state=seed+1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.7, random_state=seed+1)


scaler_X = StandardScaler()
scaler_Y = StandardScaler()
X_scaled = scaler_X.fit_transform(X_train)
y_scaled = scaler_Y.fit_transform(y_train)

batch_size = cfg.config_set["batch_size"]

dataset = LTPDataset(X_scaled, y_scaled)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


X_val_scaled = scaler_X.transform(X_val)
y_val_scaled = scaler_Y.transform(y_val)

val_dataset = LTPDataset(X_val_scaled, y_val_scaled)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

X_scaled_test = scaler_X.transform(X_test)
y_scaled_test = scaler_Y.transform(y_test)

test_dataset = LTPDataset(X_scaled_test, y_scaled_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [3]:

#### Define the Conditional Denoising Autoencoder (CDAE) model

class ConditionalDenoisingAutoencoder(nn.Module):
    def __init__(self, ouput_dim=17, cond_dim=3, latent_dim=8, noise_std=0.5):
        super(ConditionalDenoisingAutoencoder, self).__init__()
        self.noise_std = noise_std
        
        ###* Conditional Encoder
        self.encoder = nn.Sequential(
            nn.Linear(ouput_dim + cond_dim, 200), 
            nn.ReLU(),
            nn.Linear(200, latent_dim)
        )
        
        
        ###* Conditional Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + cond_dim, 200), 
            nn.ReLU(),
            nn.Linear(200, ouput_dim)
        )
    
    
    def forward(self, y, cond, add_noise=True):
        if self.training and add_noise:
            noise = torch.randn_like(y) * self.noise_std
            y_noisy = y + noise
        else:
            y_noisy = y
        
        enc_input = torch.cat([y_noisy, cond], dim=1)
        latent = self.encoder(enc_input)
        
        dec_input = torch.cat([latent, cond], dim=1)
        y_recon = self.decoder(dec_input)
        
        return y_recon, latent

In [4]:
### Instantiate the model
### best noise_std = 0.1

latent_dim = 4 
cdae = ConditionalDenoisingAutoencoder(ouput_dim=17, cond_dim=3, latent_dim=latent_dim, noise_std=0.05)


config_cdae = {
    "lr": 1e-3,
    "num_epochs": 400,
    "lambda_sparse": 8e-3,
    "p_clean": 0.7,
    "noise_std": 0.05,

    "latent_dim": 3,
}

num_epochs = config_cdae['num_epochs']
lr = config_cdae['lr']
criterion = nn.MSELoss()
optimizer_cdae = optim.Adam(cdae.parameters(), lr=lr)

lambda_sparse = config_cdae['lambda_sparse']
p_clean = config_cdae['p_clean']


In [5]:
#### Train the conditional denoising autoencoder

cdae.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for cond, y in dataloader:
        optimizer_cdae.zero_grad()
        
        if np.random.rand() < p_clean:
            add_noise = False
        else:
            add_noise = True
        
        
        y_recon, latent = cdae(y, cond, add_noise=add_noise)
        
        
        loss_sparse = torch.mean(torch.abs(latent))
        
        loss = criterion(y_recon, y) + lambda_sparse * loss_sparse
        loss.backward()
        optimizer_cdae.step()
        epoch_loss += loss.item() * cond.size(0)
    epoch_loss /= len(dataloader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs} Alternating Autoencoder Loss: {epoch_loss:.5f} Sparse Loss: {loss_sparse.item():.5f}")

Epoch 1/400 Alternating Autoencoder Loss: 0.29708 Sparse Loss: 0.91099
Epoch 2/400 Alternating Autoencoder Loss: 0.04908 Sparse Loss: 0.74067
Epoch 3/400 Alternating Autoencoder Loss: 0.02184 Sparse Loss: 1.02337
Epoch 4/400 Alternating Autoencoder Loss: 0.01350 Sparse Loss: 0.54008
Epoch 5/400 Alternating Autoencoder Loss: 0.01048 Sparse Loss: 0.44178
Epoch 6/400 Alternating Autoencoder Loss: 0.00853 Sparse Loss: 0.59197
Epoch 7/400 Alternating Autoencoder Loss: 0.00721 Sparse Loss: 0.68102
Epoch 8/400 Alternating Autoencoder Loss: 0.00655 Sparse Loss: 0.54587
Epoch 9/400 Alternating Autoencoder Loss: 0.00575 Sparse Loss: 0.43812
Epoch 10/400 Alternating Autoencoder Loss: 0.00547 Sparse Loss: 0.59769
Epoch 11/400 Alternating Autoencoder Loss: 0.00481 Sparse Loss: 0.21312
Epoch 12/400 Alternating Autoencoder Loss: 0.00466 Sparse Loss: 0.25402
Epoch 13/400 Alternating Autoencoder Loss: 0.00445 Sparse Loss: 0.26482
Epoch 14/400 Alternating Autoencoder Loss: 0.00406 Sparse Loss: 0.22943
E

In [6]:

def evaluate_model_cdae(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0.0
    y_recon_vec = []
    with torch.no_grad():
        for cond, y in dataloader:
            y_recon, _ = model(y, cond, add_noise=False)
            loss = criterion(y_recon, y)
            epoch_loss += loss.item() * cond.size(0)
        
            y_recon_vec.append(y_recon.detach().numpy())
            
    epoch_loss /= len(dataloader.dataset)
    y_recon_vec = np.concatenate(y_recon_vec, axis=0)
    
    return epoch_loss, y_recon_vec


loss_cdae, y_recon = evaluate_model_cdae(cdae, test_dataloader, criterion)
print(f"Test Loss: {loss_cdae:.5f}")

Test Loss: 0.00019


In [7]:

class MappingNetwork(nn.Module):
    def __init__(self, cond_dim=3, latent_dim=8):
        super(MappingNetwork, self).__init__()
        
        self.residual = nn.Sequential(
            nn.Linear(cond_dim, 100),
            nn.ReLU(),
            nn.Linear(100, latent_dim)
        )
    
    def forward(self, cond):
        return self.residual(cond)



In [8]:
mapping_net = MappingNetwork(cond_dim=3, latent_dim=latent_dim)

for param in cdae.decoder.parameters():
    param.requires_grad = False
for param in cdae.encoder.parameters():
    param.requires_grad = False


optimizer_mapping = optim.Adam(mapping_net.parameters(), lr=lr)
criterion = nn.MSELoss()

config_mapping = {
    "lr": 1e-3,
    "num_epochs": 300,
    "lambda_theta": 1e-3
}


num_epochs_mapping = config_mapping['num_epochs']

lambda_theta = config_mapping['lambda_theta']

mapping_net.train()
cdae.eval()

for epoch in range(num_epochs_mapping):
    epoch_loss = 0.0
    for cond, y in dataloader:
        optimizer_mapping.zero_grad()
        
        with torch.no_grad():
            _, latent = cdae(y, cond, add_noise=False)
        
        pred_latent = mapping_net(cond)
        loss_map = criterion(pred_latent, latent)
    
        loss_theta = torch.norm(mapping_net.residual[0].weight, p=2)**2 + torch.norm(mapping_net.residual[2].weight, p=2)**2
        
        total_loss = loss_map + lambda_theta * loss_theta
        total_loss.backward()
        optimizer_mapping.step()
        
        epoch_loss += total_loss.item() * cond.size(0)
    epoch_loss /= len(dataloader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs_mapping} Mapping Network Loss: {epoch_loss:.7f} Loss Theta: {loss_theta:.5f}")

Epoch 1/300 Mapping Network Loss: 0.0529444 Loss Theta: 32.67643
Epoch 2/300 Mapping Network Loss: 0.0366312 Loss Theta: 26.85452
Epoch 3/300 Mapping Network Loss: 0.0304944 Loss Theta: 21.58237
Epoch 4/300 Mapping Network Loss: 0.0253232 Loss Theta: 17.15053
Epoch 5/300 Mapping Network Loss: 0.0211441 Loss Theta: 13.53085
Epoch 6/300 Mapping Network Loss: 0.0177486 Loss Theta: 10.68656
Epoch 7/300 Mapping Network Loss: 0.0151356 Loss Theta: 8.47647
Epoch 8/300 Mapping Network Loss: 0.0130688 Loss Theta: 6.78231
Epoch 9/300 Mapping Network Loss: 0.0114903 Loss Theta: 5.48857
Epoch 10/300 Mapping Network Loss: 0.0103073 Loss Theta: 4.48259
Epoch 11/300 Mapping Network Loss: 0.0094034 Loss Theta: 3.75442
Epoch 12/300 Mapping Network Loss: 0.0087015 Loss Theta: 3.19369
Epoch 13/300 Mapping Network Loss: 0.0081854 Loss Theta: 2.77425
Epoch 14/300 Mapping Network Loss: 0.0077571 Loss Theta: 2.42163
Epoch 15/300 Mapping Network Loss: 0.0074614 Loss Theta: 2.23135
Epoch 16/300 Mapping Network

In [9]:

#### Final Predictor
class FinalPredictor(nn.Module):
    def __init__(self, mapping_net, decoder):
        super(FinalPredictor, self).__init__()
        self.mapping_net = mapping_net
        self.decoder = decoder
    
    def forward(self, cond):
        pred_latent = self.mapping_net(cond)
        
        dec_input = torch.cat([pred_latent, cond], dim=1)
        y_pred = self.decoder(dec_input)
        return y_pred


for param in cdae.decoder.parameters():
    param.requires_grad = True

final_model = FinalPredictor(mapping_net, cdae.decoder)

optimizer_final = optim.Adam(final_model.parameters(), lr=lr)



config_final = {
    "num_epochs":200, # 300
    "lr": 1e-3,
    "p_encoder": 0.0,
    "max_norm_clip": 0.1
}


num_epochs_final = config_final['num_epochs']
p_encoder = config_final['p_encoder']
max_norm_clip = config_final['max_norm_clip']


final_model.train()
for epoch in range(num_epochs_final):
    epoch_loss = 0.0
    for cond, y in dataloader:
        optimizer_final.zero_grad()
        
        if np.random.rand() < p_encoder:
            with torch.no_grad():
                _, latent = cdae(y, cond, add_noise=False)
        else:
            latent = final_model.mapping_net(cond)
        
        dec_input = torch.cat([latent, cond], dim=1)
        y_pred = final_model.decoder(dec_input)
        loss = criterion(y_pred, y)
        
        ### clip loss
        torch.nn.utils.clip_grad_norm_(final_model.parameters(), max_norm_clip)
        
        loss.backward()
        optimizer_final.step()
        epoch_loss += loss.item() * cond.size(0)
    epoch_loss /= len(dataloader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs_final} Final Model Loss: {epoch_loss:.5f}")



Epoch 1/200 Final Model Loss: 0.04945
Epoch 2/200 Final Model Loss: 0.04301
Epoch 3/200 Final Model Loss: 0.03883
Epoch 4/200 Final Model Loss: 0.03536
Epoch 5/200 Final Model Loss: 0.03244
Epoch 6/200 Final Model Loss: 0.02891
Epoch 7/200 Final Model Loss: 0.02582
Epoch 8/200 Final Model Loss: 0.02440
Epoch 9/200 Final Model Loss: 0.02178
Epoch 10/200 Final Model Loss: 0.02026
Epoch 11/200 Final Model Loss: 0.01828
Epoch 12/200 Final Model Loss: 0.01683
Epoch 13/200 Final Model Loss: 0.01544
Epoch 14/200 Final Model Loss: 0.01452
Epoch 15/200 Final Model Loss: 0.01387
Epoch 16/200 Final Model Loss: 0.01396
Epoch 17/200 Final Model Loss: 0.01166
Epoch 18/200 Final Model Loss: 0.01142
Epoch 19/200 Final Model Loss: 0.00994
Epoch 20/200 Final Model Loss: 0.00941
Epoch 21/200 Final Model Loss: 0.00880
Epoch 22/200 Final Model Loss: 0.00853
Epoch 23/200 Final Model Loss: 0.00841
Epoch 24/200 Final Model Loss: 0.00813
Epoch 25/200 Final Model Loss: 0.00744
Epoch 26/200 Final Model Loss: 0.0

In [10]:
def evaluate_final_model(model, dataloader, criterion):
    
    model.eval()
    total_loss = 0.0
    total_samples = 0
    
    all_preds = []
    all_truth = []
    
    with torch.no_grad():
        for cond, y in dataloader:
            y_pred = model(cond)
            
            all_preds.append(y_pred)
            all_truth.append(y)
            
            loss = criterion(y_pred, y)
            total_loss += loss.item() * cond.size(0)
            total_samples += cond.size(0)
            
    avg_loss = total_loss / total_samples
    
    all_preds = torch.cat(all_preds, dim=0)
    all_truth = torch.cat(all_truth, dim=0)
    
    error_per_component = torch.abs(all_preds - all_truth)
    max_error_per_component = torch.max(error_per_component, dim=0).values
    min_error_per_component = torch.min(error_per_component, dim=0).values
    median_error_per_component = torch.median(error_per_component, dim=0).values
    
    mse_per_component = torch.mean((all_preds - all_truth)**2, dim=0)
    rsme_per_component = torch.sqrt(mse_per_component)

    return avg_loss, rsme_per_component, max_error_per_component, min_error_per_component, median_error_per_component, error_per_component


test_loss, rsme_per_component, max_per_component, min_per_component, median_per_component, error_per_component = evaluate_final_model(final_model, test_dataloader, criterion)
print(f"RMSQ Test Loss: {np.sqrt(test_loss):.5f}")
print(f"Test Loss: {test_loss:.5f}")
# print(rsme_per_component)
# print(max_per_component)
# print(min_per_component)
# print(median_per_component)


RMSQ Test Loss: 0.03636
Test Loss: 0.00132


In [12]:


print(f"Number of parameters in the final model: {count_parameters(final_model)}")

Number of parameters in the final model: 5821
